<h2>The missionaries and cannibals problem is usually stated as follows. <br>
<ul>
<li>Three missionaries and three cannibals are on one side of a river, along with a boat that can hold one or
two people. 
<li>Find a way to get everyone to the other side without ever leaving a group of missionaries
in one place outnumbered by the cannibals in that place.
<li>The boat cannot cross the river by itself with no people on board.

In [38]:
import copy

# a node represents a state which is the result of an action from a previous node(state)
#
# for the current node, compute it's next moves and generate nodes for these
#  each has a move(that led to it as a state), a parent (the node that generated it)
#  - these nodes are the frontier, known action with unexpanded/unknown states
#  - put the frontier in a queue
#
# then in queue order we expand each frontier node at the current
#  - expanding means performing the transfer action and generating a new resulting state in the current node
#   - state should be the state of both sides
#  - expanding can lead to a success (a state where the node is the dest and has ALL )
#  - expanding can lead to a dead end (state where we can't transfer people from the current side without reverting to the previous state)
#  - expanding can lead to a new set of actions(new frontier)

class node:
    """A single node in a tree for the missionary/cannibal problem. Encapsulates
        a state which results from making a move from a previous node.
    
        Attributes:
            children(list[node]): nodes which represent the result of this nodes next moves
            parent(node): node whose action led to this node.  
                - If null we're at the root node.
            current_side: side of the river in which we are in this node
            state(dict): list of people on each side of the river, and which side we're on
    """
    def __init__(self, children, parent, state):
        self.children = children
        self._parent = parent
        self.state = state
        
    def get_parent(self):
        return self._parent
    
    def get_current_side(self):
        return self.state[SIDE]
    
    def is_root(self):
        return True if self._parent is None else False
    
    def get_current_people(self):
        """Get a list of the people on the current side
        """
        if self.state[SIDE] == ORIG:
            return self.state[ORIG]
        else:
            return self.state[DEST]
    
    def get_next_moves(self):
        """Return a list of all the possible next moves from the current side
        """
        next_moves = []
        current_people = self.get_current_people()

        if self.get_current_side() == ORIG:
            if not (MISSIONARY in current_people) and not (CANNIBAL in current_people):
                raise ValueError('There are no people on the current side to move to the other side')
            if MISSIONARY in current_people:
                next_moves += MISSIONARY
            if len([i for i, x in enumerate(current_people) if x == MISSIONARY]) >= 2:
                next_moves.append([MISSIONARY, MISSIONARY])
            if CANNIBAL in current_people:
                next_moves += CANNIBAL
            if len([i for i, x in enumerate(current_people) if x == CANNIBAL]) >= 2:
                next_moves.append([CANNIBAL, CANNIBAL])
            if MISSIONARY in current_people and CANNIBAL in current_people:
                next_moves.append([MISSIONARY, CANNIBAL])
        elif self.get_current_side() == DEST:
            # This case would lead to a situation where we're besically returning to the 
            # previous state and so serves as an end point, one with no more moves.
            if len(current_people) == 1:
                return []
            if not (MISSIONARY in current_people) and not (CANNIBAL in current_people):
                raise ValueError('There are no people on the current side to move to the other side')
            if MISSIONARY in current_people:
                next_moves += MISSIONARY
            # similar to above, no point in moving all existing people from the dest side
            if (len(current_people) != 2) and len([i for i, x in enumerate(current_people) if x == MISSIONARY]) >= 2:
                next_moves.append([MISSIONARY, MISSIONARY])
            if CANNIBAL in current_people:
                next_moves += CANNIBAL
            # similar to above, no point in moving all existing people from the dest side
            if (len(current_people) != 2) and len([i for i, x in enumerate(current_people) if x == CANNIBAL]) >= 2:
                next_moves.append([CANNIBAL, CANNIBAL])
            # similar to above, no point in moving all existing people from the dest side
            if (len(current_people) != 2) and MISSIONARY in current_people and CANNIBAL in current_people:
                next_moves.append([MISSIONARY, CANNIBAL])

        return next_moves

    def is_goal_state(self):
        GOAL_STATE = ["M", "M", "M", "C", "C", "C"]
        
        if (self.state[DEST] == GOAL_STATE):
            if (self.state[ORIG] != []):
                print("In goal state but somehow ORIG side still has people.  Should not happen!!!")
            return True
        else:
            return False

    def __str__(self):
        return_string = "Node:"
        if self.is_root():
            return_string += "\n  Is Root"
        else:
            return_string += "\n  parent state: {0}".format(self.get_parent().state)
            
        return_string += "\n  state: {0}".format(self.state)
        return_string += "\n  num children: {0}".format(len(self.children))
        return_string += "\n"
        return return_string
        
    __repr__ = __str__
    

def get_other_side(current_side):
    """Get a list of the people on the current side
    """
    if current_side == 'ORIG':
        return DEST
    else:
        return ORIG

def expand(node_obj):
    """Creation and attachment of new nodes with new state 
        from current state and the next move being made.  
        These new nodes represent the 'frontier' or the next 
        possible states which have not themselves been explored


        Args:
            node_obj: the node to expand.
        Return:
            list: the new nodes
    """

    next_moves = node_obj.get_next_moves()
    next_move_nodes = []

    for next_move in next_moves:
        prev_state = copy.deepcopy(node_obj.state)
        prev_side = prev_state[SIDE]
        new_side = get_other_side(node_obj.get_current_side())

        # remove from the previous side and add to the new side
        prev_side_people = prev_state[prev_side]
        new_side_people = prev_state[new_side]

        for next_move_item in next_move:
            prev_side_people.remove(next_move_item)
            new_side_people.append(next_move_item)
                    
        new_state = {prev_state[SIDE]: prev_side_people, 
                     new_side:new_side_people,
                     SIDE:new_side}
        next_move_node = node([], node_obj, new_state)
        next_move_nodes.append(next_move_node)
        
        
    node_obj.children = next_move_nodes    
    return next_move_nodes

def states_are_equal(state1, state2):
    """Equality check for states which ignores order.
        For instance these states shouuld be equal :
        {'side': 'ORIG', 'ORIG': ['M', 'C', 'C', 'C', 'M', 'M'], 'DEST': []} and 
        {'side': 'ORIG', 'ORIG': ['M', 'M', 'M', 'C', 'C', 'C'], 'DEST': []} 
    """
    if ((state1[SIDE] == state2[SIDE])
        and (sorted(state1[ORIG]) == sorted(state2[ORIG]))
        and (sorted(state1[DEST]) == sorted(state2[DEST]))):
        return True
    else:
        return False
    
def state_processed(states, compare_state):
    if len([state for state in states if states_are_equal(state, compare_state)]) != 0:
        return True
    else:
        return False
    
def state_in_frontier(frontier, compare_state):
    if len([node for node in frontier if states_are_equal(node.state, compare_state)]) != 0:
        return True
    else:
        return False

def state_is_invalid(state):
    """No side should have more cannibals than missionaries
    """
    return True if ((state[ORIG].count(MISSIONARY) > 0 and (state[ORIG].count(MISSIONARY) < state[ORIG].count(CANNIBAL)))
                    or (state[DEST].count(MISSIONARY) > 0 and (state[DEST].count(MISSIONARY) < state[DEST].count(CANNIBAL)))) else False

In [39]:
#
# Build the solution tree
#
import pandas as pd

SIDE = 'current_side'
ORIG = 'ORIG'
DEST = 'DEST'
MISSIONARY = 'M'
CANNIBAL = 'C'
# queue of nodes to be processed
frontier = []
# list of states(dict) which have been expanded
processed_states = []
MAX_LOOP_COUNT = 15
done = False
loop_count = 1

# initial state with no children or parent
root_node = node([], None, { ORIG:[MISSIONARY, MISSIONARY, MISSIONARY, CANNIBAL, CANNIBAL, CANNIBAL], 
                    DEST:[],
                    SIDE: ORIG})
frontier.append(root_node)

if root_node.is_goal_state():
    print("GOAL STATE FOUND at root")
else:
    while not(done) and (loop_count <= MAX_LOOP_COUNT):
        
        if frontier == []:
            raise StopIteration("frontier is Empty and should not be")
            
        next_node = frontier.pop(0)
        processed_states.append(next_node.state.copy())
        # expand this node, append new nodes on frontier
        children_nodes = expand(next_node)
        
        for child_node in children_nodes:

            if child_node.is_goal_state():
                GOAL_NODE = copy.deepcopy(child_node)
                done = True
            elif not (state_is_invalid(child_node.state) 
                      or state_in_frontier(frontier, child_node.state)
                      or state_processed(processed_states, child_node.state)
                     ):
                frontier.append(child_node)
                
        loop_count += 1

In [41]:
#
# Print the solution tree
#

GOAL_NODE.get_parent().get_parent().get_parent()

"""Start with goal node and isert each parent till we 
    have the entire path that led to the goal
"""
back_trace_done = False
back_trace_loop_count = 1
back_trace_max_loop_count = 15
result_list = [GOAL_NODE]

current_node = GOAL_NODE

while (not back_trace_done) and (back_trace_loop_count < back_trace_max_loop_count):
    if current_node.is_root():
        back_trace_done = True
    else:
        current_node = current_node.get_parent()
        result_list.insert(0, current_node)
        back_trace_loop_count += 1
        
print("Turns to solution = {0}\n".format(back_trace_loop_count-1))

print("Solution :")
for result in result_list:
    print(result.state)

Turns to solution = 11

Solution :
{'ORIG': ['M', 'M', 'M', 'C', 'C', 'C'], 'DEST': [], 'current_side': 'ORIG'}
{'ORIG': ['M', 'M', 'M', 'C'], 'DEST': ['C', 'C'], 'current_side': 'DEST'}
{'DEST': ['C'], 'ORIG': ['M', 'M', 'M', 'C', 'C'], 'current_side': 'ORIG'}
{'ORIG': ['M', 'M', 'M'], 'DEST': ['C', 'C', 'C'], 'current_side': 'DEST'}
{'DEST': ['C', 'C'], 'ORIG': ['M', 'M', 'M', 'C'], 'current_side': 'ORIG'}
{'ORIG': ['M', 'C'], 'DEST': ['C', 'C', 'M', 'M'], 'current_side': 'DEST'}
{'DEST': ['C', 'M'], 'ORIG': ['M', 'C', 'M', 'C'], 'current_side': 'ORIG'}
{'ORIG': ['C', 'C'], 'DEST': ['C', 'M', 'M', 'M'], 'current_side': 'DEST'}
{'DEST': ['M', 'M', 'M'], 'ORIG': ['C', 'C', 'C'], 'current_side': 'ORIG'}
{'ORIG': ['C'], 'DEST': ['M', 'M', 'M', 'C', 'C'], 'current_side': 'DEST'}
{'DEST': ['M', 'M', 'M', 'C'], 'ORIG': ['C', 'C'], 'current_side': 'ORIG'}
{'ORIG': [], 'DEST': ['M', 'M', 'M', 'C', 'C', 'C'], 'current_side': 'DEST'}
